In [7]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import  sklearn.linear_model as linearModel
from sklearn import preprocessing
from sklearn.manifold import TSNE
import numpy as np
#import seaborn
import matplotlib.pyplot as plt
from itertools import cycle
from warnings import filterwarnings
import timeit
import pymc3 as pm
import sys
import json
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression


In [8]:
sys.path.insert(0,'./src')
import bridgeSampling as bs # it contains a method to estimate the marginal likelihood according to the Bridge Sampling approach
import utilFunctions as ut          # it has different methods to handle and plot data
import BayesianModels as bm # it has methods to build and train bayesian model (Logistic Regression and Neural Nets)
import experiments as exp
import samplebiasselection as sbs

In [19]:
number = 3
fileTrain = 'data/BIASEDFATS/Train_rrlyr-'+str(number)+'.csv'
fileTest = 'data/BIASEDFATS/Test_rrlyr-'+str(number)+'.csv'

# Basic Kernel Ridge Logistic Regresion (Polynomial)

In [20]:
Cvalue = 1
plot = False
basicLR_stats_acc = []
basicLR_stats_f1 = []
res = []
for k in [1, 2, 3]:
    print('kernel '+str(k))
    for components in [2, 4, 6, 8, 10, 12, 14]:
        for size in[500, 1000, 10000, 20000, 50000]:
            print('components '+str(components))
            dataTrain = pd.read_csv(fileTrain)
            dataTest = pd.read_csv(fileTest)
            print(len(dataTrain.columns)) 
            try:
                dataTrain = dataTrain.loc[:, ~dataTrain.columns.str.contains('^Unnamed')]
                dataTrain =  dataTrain.drop(['Pred', 'Pred2', 'h', 'e', 'u','ID'], axis = 1)
                dataTrain = dataTrain.loc[:, (dataTrain != dataTrain.iloc[0]).any()]
                dataTest = dataTest[list(dataTrain.columns)]

            except:
                print('---')
            dataTrain = ut.downSampling(dataTrain)
            try:
                dataTrain = dataTrain.sample(size, random_state = 0)
            except:
                print('sample bigger that population')
            yTrain = 1*(dataTrain['label'] == 'ClassA')
            yTestHO = 1*(dataTest['label'] == 'ClassA')
            del dataTrain['label']
            del dataTest['label']

            names = dataTrain.columns
            scaler = preprocessing.StandardScaler()
            dataTrain = scaler.fit_transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain, columns=names)

            dataTest = scaler.fit_transform(dataTest)
            dataTest = pd.DataFrame(dataTest, columns=names)


            pca = PCA(n_components=components)
            pca.fit(dataTrain)
            dataTrain = pca.transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain)
            dataTrain = ut.kernelPolinomial(dataTrain,k)


            pca.fit(dataTest)
            dataTest = pca.transform(dataTest)
            dataTest = pd.DataFrame(dataTest)
            dataTest = ut.kernelPolinomial(dataTest,k)
            #size = dataTrain.shape[0]

            start_1 = timeit.default_timer()
            skf = StratifiedKFold(n_splits=int(10))
            skf.get_n_splits(dataTrain, yTrain)
            acc_kfold_Train = []
            f1_kfold_Train = []
            i = 0

            for train_index, test_index in skf.split(dataTrain, yTrain):
                        i = i+1
                        print('training fold ' + str(i))
                        X_train, X_test = dataTrain.iloc[train_index,:], dataTrain.iloc[test_index,:]
                        y_train, y_test = yTrain.iloc[train_index], yTrain.iloc[test_index]

                        basicLR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial',
                                                     C=Cvalue).fit(X_train, y_train)
                        predictions_1_Train = basicLR.predict(X_test)

                        accTrain = accuracy_score(y_test, predictions_1_Train, normalize=True)
                        f1Train = f1_score(y_test, predictions_1_Train, pos_label = 1)

                        if plot:
                            cm = confusion_matrix(y_test, predictions_1_Train)
                            ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')
                        print('Accuracy train: ', accTrain)
                        #print('Accuracy f1 Train: ', f1Train)
                        basicLR_stats_acc.append(accTrain)
                        basicLR_stats_f1.append(f1Train)
            accTrain = np.mean(basicLR_stats_acc)
            f1Train =np.mean(basicLR_stats_f1)
            print('----------------------------------------------------------')
            print('Summary Training')
            print('Mean Accuracy train: ', accTrain)
            print('Mean f1 Train: ', f1Train)
            stop_1 = timeit.default_timer()
            time_CV = stop_1 - start_1
            basicLR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', C=Cvalue).fit(dataTrain, yTrain)
            predictions_1_Test = basicLR.predict(dataTest)
            print(predictions_1_Test.shape)
            print(yTestHO.shape)
            print('----------------------------------------------------------')
            print('Summary Testing')
            accTest = accuracy_score(yTestHO, predictions_1_Test, normalize=True)
            f1Test = f1_score(yTestHO, predictions_1_Test,pos_label = 1)
            if plot: 
                cm = confusion_matrix(yTestHO, predictions_1_Test)
                ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')

            print(' Accuracy test: ', accTest)
            print(' f1 test: ', f1Test)
            print('----------------------------------------------------------')

            res.append([k, components, size,  accTrain, accTest, f1Train, f1Test])
            pd.DataFrame(res).to_csv('Results/summaryMCMC/'+str(number)+'-BasicCV1710C'+str(Cvalue)+'.csv')

kernel 1
components 2
71
training fold 1
Accuracy train:  0.9215686274509803
training fold 2
Accuracy train:  0.8823529411764706
training fold 3
Accuracy train:  0.8627450980392157
training fold 4
Accuracy train:  0.9607843137254902
training fold 5
Accuracy train:  0.88
training fold 6
Accuracy train:  0.9
training fold 7
Accuracy train:  0.8163265306122449
training fold 8
Accuracy train:  0.8775510204081632
training fold 9
Accuracy train:  0.8163265306122449
training fold 10
Accuracy train:  0.8775510204081632
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.8795206082432973
Mean f1 Train:  0.8821269860917635
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6584799478703868
 f1 test:  0.3653736075029721
----------------------------------------------------------
components 2
71
training fold 1
Accuracy train:  0.900990099009901
training fold 2
Accuracy train:  0.92
training

(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7829986375214738
 f1 test:  0.48648648648648646
----------------------------------------------------------
components 6
71
training fold 1
Accuracy train:  0.9607843137254902
training fold 2
Accuracy train:  0.9411764705882353
training fold 3
Accuracy train:  1.0
training fold 4
Accuracy train:  0.9803921568627451
training fold 5
Accuracy train:  0.96
training fold 6
Accuracy train:  0.94
training fold 7
Accuracy train:  0.9387755102040817
training fold 8
Accuracy train:  0.9591836734693877
training fold 9
Accuracy train:  0.8571428571428571
training fold 10
Accuracy train:  0.9183673469387755
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9151367312440538
Mean f1 Train:  0.916866594969095
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7274924471299093
 f1 tes

Accuracy train:  0.9545909181836367
training fold 9
Accuracy train:  0.9563912782556512
training fold 10
Accuracy train:  0.9459891978395679
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9311855826193384
Mean f1 Train:  0.9328612831664153
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.8003909720988093
 f1 test:  0.5080588647512263
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.9215686274509803
training fold 2
Accuracy train:  0.9411764705882353
training fold 3
Accuracy train:  0.9803921568627451
training fold 4
Accuracy train:  0.9803921568627451
training fold 5
Accuracy train:  0.88
training fold 6
Accuracy train:  0.94
training fold 7
Accuracy train:  0.9387755102040817
training fold 8
Accuracy train:  0.9591836734693877
training fold 9
Accuracy train:  0.8775510204081632
training fold 10
Accuracy train:

Accuracy train:  0.9544091181763648
training fold 2
Accuracy train:  0.9618076384723055
training fold 3
Accuracy train:  0.9568086382723455
training fold 4
Accuracy train:  0.9654069186162767
training fold 5
Accuracy train:  0.9658
training fold 6
Accuracy train:  0.9588
training fold 7
Accuracy train:  0.9595919183836767
training fold 8
Accuracy train:  0.9603920784156831
training fold 9
Accuracy train:  0.9615923184636928
training fold 10
Accuracy train:  0.95499099819964
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9377432556093135
Mean f1 Train:  0.9393602438476846
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.8172620105443991
 f1 test:  0.5302716530637106
----------------------------------------------------------
components 14
71
training fold 1
Accuracy train:  0.9803921568627451
training fold 2
Accuracy train:  0.9215686274509803
training fold 3
Accuracy train

71
training fold 1
Accuracy train:  0.9010197960407919
training fold 2
Accuracy train:  0.9122175564887023
training fold 3
Accuracy train:  0.9062187562487503
training fold 4
Accuracy train:  0.9110177964407119
training fold 5
Accuracy train:  0.9054
training fold 6
Accuracy train:  0.907
training fold 7
Accuracy train:  0.9041808361672334
training fold 8
Accuracy train:  0.908381676335267
training fold 9
Accuracy train:  0.9127825565113022
training fold 10
Accuracy train:  0.9047809561912382
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9363090069039928
Mean f1 Train:  0.9382340859623812
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7112730288490018
 f1 test:  0.4066806252130301
----------------------------------------------------------
components 4
71
training fold 1
Accuracy train:  0.9607843137254902
training fold 2
Accuracy train:  0.9215686274509803
training fol

...

Accuracy train:  0.9729864932466233
training fold 9
Accuracy train:  0.9749874937468734
training fold 10
Accuracy train:  0.9784892446223111
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9444516259721032
Mean f1 Train:  0.9461968501873501
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7735086783958296
 f1 test:  0.4768915036256669
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.9640071985602879
training fold 2
Accuracy train:  0.9716056788642271
training fold 3
Accuracy train:  0.9648070385922816
training fold 4
Accuracy train:  0.9758048390321936
training fold 5
Accuracy train:  0.975
training fold 6
Accuracy train:  0.9672
training fold 7
Accuracy train:  0.9665933186637328
training fold 8
Accuracy train:  0.967993598719744
training fold 9
Accuracy train:  0.9731946389277856
training fold 10
Accuracy trai

71
training fold 1
Accuracy train:  0.9685157421289355
training fold 2
Accuracy train:  0.9650174912543729
training fold 3
Accuracy train:  0.9720139930034982
training fold 4
Accuracy train:  0.9675
training fold 5
Accuracy train:  0.973
training fold 6
Accuracy train:  0.9645
training fold 7
Accuracy train:  0.9675
training fold 8
Accuracy train:  0.9739869934967483
training fold 9
Accuracy train:  0.9754877438719359
training fold 10
Accuracy train:  0.9769884942471235
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9472916322534818
Mean f1 Train:  0.9489715510352177
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7645637106806469
 f1 test:  0.46723860589812327
----------------------------------------------------------
components 14
71
training fold 1
Accuracy train:  0.967006598680264
training fold 2
Accuracy train:  0.9716056788642271
training fold 3
Accuracy train:  0

(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7846217641135004
 f1 test:  0.4894543207796219
----------------------------------------------------------
components 4
71
training fold 1
Accuracy train:  0.9590204897551224
training fold 2
Accuracy train:  0.953023488255872
training fold 3
Accuracy train:  0.9580209895052474
training fold 4
Accuracy train:  0.963
training fold 5
Accuracy train:  0.9685
training fold 6
Accuracy train:  0.955
training fold 7
Accuracy train:  0.9595
training fold 8
Accuracy train:  0.9674837418709354
training fold 9
Accuracy train:  0.9704852426213106
training fold 10
Accuracy train:  0.9674837418709354
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9457555517344743
Mean f1 Train:  0.9476007034844318
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7727148865588531
 f1 test:  0.47

Accuracy train:  0.973973973973974
training fold 10
Accuracy train:  0.972972972972973
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9474044058274935
Mean f1 Train:  0.9492086769297479
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7686156033410343
 f1 test:  0.4716766758643078
----------------------------------------------------------
components 8
71
training fold 1
Accuracy train:  0.9665167416291854
training fold 2
Accuracy train:  0.9610194902548725
training fold 3
Accuracy train:  0.9690154922538731
training fold 4
Accuracy train:  0.9665
training fold 5
Accuracy train:  0.9705
training fold 6
Accuracy train:  0.9595
training fold 7
Accuracy train:  0.961
training fold 8
Accuracy train:  0.9724862431215607
training fold 9
Accuracy train:  0.9734867433716858
training fold 10
Accuracy train:  0.967983991995998
--------------------------------------------------------

Accuracy train:  0.945054945054945
training fold 4
Accuracy train:  0.958
training fold 5
Accuracy train:  0.958
training fold 6
Accuracy train:  0.964
training fold 7
Accuracy train:  0.964
training fold 8
Accuracy train:  0.9579579579579579
training fold 9
Accuracy train:  0.9679679679679679
training fold 10
Accuracy train:  0.9719719719719719
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.948759004127043
Mean f1 Train:  0.950528240654548
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7624903737930218
 f1 test:  0.46479963691699816
----------------------------------------------------------
components 12
71
training fold 1
Accuracy train:  0.95952023988006
training fold 2
Accuracy train:  0.9490254872563718
training fold 3
Accuracy train:  0.95952023988006
training fold 4
Accuracy train:  0.9635
training fold 5
Accuracy train:  0.966
training fold 6
Accuracy train:  0.

In [21]:
Cvalue = 100
plot = False
basicLR_stats_acc = []
basicLR_stats_f1 = []
res = []
for k in [1, 2, 3]:
    print('kernel '+str(k))
    for components in [2, 4, 6, 8, 10, 12, 14]:
        for size in[500, 1000, 10000, 20000, 50000]:
            print('components '+str(components))
            dataTrain = pd.read_csv(fileTrain)
            dataTest = pd.read_csv(fileTest)
            print(len(dataTrain.columns)) 
            try:
                dataTrain = dataTrain.loc[:, ~dataTrain.columns.str.contains('^Unnamed')]
                dataTrain =  dataTrain.drop(['Pred', 'Pred2', 'h', 'e', 'u','ID'], axis = 1)
                dataTrain = dataTrain.loc[:, (dataTrain != dataTrain.iloc[0]).any()]
                dataTest = dataTest[list(dataTrain.columns)]

            except:
                print('---')
            dataTrain = ut.downSampling(dataTrain)
            try:
                dataTrain = dataTrain.sample(size, random_state = 0)
            except:
                print('sample bigger that population')
            yTrain = 1*(dataTrain['label'] == 'ClassA')
            yTestHO = 1*(dataTest['label'] == 'ClassA')
            del dataTrain['label']
            del dataTest['label']

            names = dataTrain.columns
            scaler = preprocessing.StandardScaler()
            dataTrain = scaler.fit_transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain, columns=names)

            dataTest = scaler.fit_transform(dataTest)
            dataTest = pd.DataFrame(dataTest, columns=names)


            pca = PCA(n_components=components)
            pca.fit(dataTrain)
            dataTrain = pca.transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain)
            dataTrain = ut.kernelPolinomial(dataTrain,k)


            pca.fit(dataTest)
            dataTest = pca.transform(dataTest)
            dataTest = pd.DataFrame(dataTest)
            dataTest = ut.kernelPolinomial(dataTest,k)
            #size = dataTrain.shape[0]

            start_1 = timeit.default_timer()
            skf = StratifiedKFold(n_splits=int(10))
            skf.get_n_splits(dataTrain, yTrain)
            acc_kfold_Train = []
            f1_kfold_Train = []
            i = 0

            for train_index, test_index in skf.split(dataTrain, yTrain):
                        i = i+1
                        print('training fold ' + str(i))
                        X_train, X_test = dataTrain.iloc[train_index,:], dataTrain.iloc[test_index,:]
                        y_train, y_test = yTrain.iloc[train_index], yTrain.iloc[test_index]

                        basicLR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial',
                                                     C=Cvalue).fit(X_train, y_train)
                        predictions_1_Train = basicLR.predict(X_test)

                        accTrain = accuracy_score(y_test, predictions_1_Train, normalize=True)
                        f1Train = f1_score(y_test, predictions_1_Train, pos_label = 1)

                        if plot:
                            cm = confusion_matrix(y_test, predictions_1_Train)
                            ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')
                        print('Accuracy train: ', accTrain)
                        #print('Accuracy f1 Train: ', f1Train)
                        basicLR_stats_acc.append(accTrain)
                        basicLR_stats_f1.append(f1Train)
            accTrain = np.mean(basicLR_stats_acc)
            f1Train =np.mean(basicLR_stats_f1)
            print('----------------------------------------------------------')
            print('Summary Training')
            print('Mean Accuracy train: ', accTrain)
            print('Mean f1 Train: ', f1Train)
            stop_1 = timeit.default_timer()
            time_CV = stop_1 - start_1
            basicLR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', C=Cvalue).fit(dataTrain, yTrain)
            predictions_1_Test = basicLR.predict(dataTest)
            print(predictions_1_Test.shape)
            print(yTestHO.shape)
            print('----------------------------------------------------------')
            print('Summary Testing')
            accTest = accuracy_score(yTestHO, predictions_1_Test, normalize=True)
            f1Test = f1_score(yTestHO, predictions_1_Test,pos_label = 1)
            if plot: 
                cm = confusion_matrix(yTestHO, predictions_1_Test)
                ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')

            print(' Accuracy test: ', accTest)
            print(' f1 test: ', f1Test)
            print('----------------------------------------------------------')

            res.append([k, components, size,  accTrain, accTest, f1Train, f1Test])
            pd.DataFrame(res).to_csv('Results/summaryMCMC/'+str(number)+'-BasicCV1710C'+str(Cvalue)+'.csv')

kernel 1
components 2
71
training fold 1
Accuracy train:  0.9215686274509803
training fold 2
Accuracy train:  0.8823529411764706
training fold 3
Accuracy train:  0.8627450980392157
training fold 4
Accuracy train:  0.9607843137254902
training fold 5
Accuracy train:  0.88
training fold 6
Accuracy train:  0.9
training fold 7
Accuracy train:  0.8163265306122449
training fold 8
Accuracy train:  0.8775510204081632
training fold 9
Accuracy train:  0.8163265306122449
training fold 10
Accuracy train:  0.8775510204081632
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.8795206082432973
Mean f1 Train:  0.8821269860917635
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6585036431491026
 f1 test:  0.3653338030649991
----------------------------------------------------------
components 2
71
training fold 1
Accuracy train:  0.900990099009901
training fold 2
Accuracy train:  0.92
training

(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7829512469640424
 f1 test:  0.4864319354115273
----------------------------------------------------------
components 6
71
training fold 1
Accuracy train:  0.9607843137254902
training fold 2
Accuracy train:  0.9411764705882353
training fold 3
Accuracy train:  1.0
training fold 4
Accuracy train:  0.9803921568627451
training fold 5
Accuracy train:  0.96
training fold 6
Accuracy train:  0.94
training fold 7
Accuracy train:  0.9183673469387755
training fold 8
Accuracy train:  0.9591836734693877
training fold 9
Accuracy train:  0.8367346938775511
training fold 10
Accuracy train:  0.9183673469387755
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9144774714561114
Mean f1 Train:  0.916228924575551
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7413897280966767
 f1 test

Accuracy train:  0.9545909181836367
training fold 9
Accuracy train:  0.9561912382476495
training fold 10
Accuracy train:  0.9459891978395679
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9306644146890423
Mean f1 Train:  0.9323746043049244
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.8002014098690836
 f1 test:  0.507821620359561
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.9215686274509803
training fold 2
Accuracy train:  0.9411764705882353
training fold 3
Accuracy train:  0.9803921568627451
training fold 4
Accuracy train:  0.9607843137254902
training fold 5
Accuracy train:  0.88
training fold 6
Accuracy train:  0.94
training fold 7
Accuracy train:  0.9387755102040817
training fold 8
Accuracy train:  0.9591836734693877
training fold 9
Accuracy train:  0.8775510204081632
training fold 10
Accuracy train: 

Accuracy train:  0.9630073985202959
training fold 3
Accuracy train:  0.9574085182963408
training fold 4
Accuracy train:  0.9662067586482703
training fold 5
Accuracy train:  0.9656
training fold 6
Accuracy train:  0.9588
training fold 7
Accuracy train:  0.9593918783756752
training fold 8
Accuracy train:  0.9595919183836767
training fold 9
Accuracy train:  0.9621924384876975
training fold 10
Accuracy train:  0.9567913582716543
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9371584460620552
Mean f1 Train:  0.9387935982033078
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.8174634204134826
 f1 test:  0.5305177194746625
----------------------------------------------------------
components 14
71
training fold 1
Accuracy train:  0.9411764705882353
training fold 2
Accuracy train:  0.9215686274509803
training fold 3
Accuracy train:  0.9607843137254902
training fold 4
Accuracy tra

(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7026005568390499
 f1 test:  0.4060947333554157
----------------------------------------------------------
components 2
71
training fold 1
Accuracy train:  0.9010197960407919
training fold 2
Accuracy train:  0.9122175564887023
training fold 3
Accuracy train:  0.9062187562487503
training fold 4
Accuracy train:  0.9110177964407119
training fold 5
Accuracy train:  0.9054
training fold 6
Accuracy train:  0.907
training fold 7
Accuracy train:  0.9041808361672334
training fold 8
Accuracy train:  0.908381676335267
training fold 9
Accuracy train:  0.9127825565113022
training fold 10
Accuracy train:  0.9047809561912382
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9356290779326941
Mean f1 Train:  0.9375663699235246
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7113204

...

training fold 1
Accuracy train:  0.9665167416291854
training fold 2
Accuracy train:  0.9650174912543729
training fold 3
Accuracy train:  0.9725137431284357
training fold 4
Accuracy train:  0.969
training fold 5
Accuracy train:  0.974
training fold 6
Accuracy train:  0.961
training fold 7
Accuracy train:  0.9665
training fold 8
Accuracy train:  0.9729864932466233
training fold 9
Accuracy train:  0.9754877438719359
training fold 10
Accuracy train:  0.9784892446223111
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9435573521570222
Mean f1 Train:  0.9452768890996587
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7738759552159232
 f1 test:  0.4772963794708879
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.9640071985602879
training fold 2
Accuracy train:  0.9718056388722256
training fold 3
Accuracy train:  0.9648

(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7794324980747586
 f1 test:  0.483621335256428
----------------------------------------------------------
components 14
71
training fold 1
Accuracy train:  0.9690154922538731
training fold 2
Accuracy train:  0.9640179910044977
training fold 3
Accuracy train:  0.9720139930034982
training fold 4
Accuracy train:  0.9685
training fold 5
Accuracy train:  0.9725
training fold 6
Accuracy train:  0.963
training fold 7
Accuracy train:  0.9675
training fold 8
Accuracy train:  0.9754877438719359
training fold 9
Accuracy train:  0.9754877438719359
training fold 10
Accuracy train:  0.9769884942471235
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9461157116113895
Mean f1 Train:  0.9477362738426672
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7624429832355903
 f1 test:  0.

Accuracy train:  0.9579579579579579
training fold 9
Accuracy train:  0.96996996996997
training fold 10
Accuracy train:  0.96996996996997
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9444752323617253
Mean f1 Train:  0.946278381106193
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7853444701143297
 f1 test:  0.4902942665841445
----------------------------------------------------------
components 4
71
training fold 1
Accuracy train:  0.95952023988006
training fold 2
Accuracy train:  0.953023488255872
training fold 3
Accuracy train:  0.9585207396301849
training fold 4
Accuracy train:  0.963
training fold 5
Accuracy train:  0.9685
training fold 6
Accuracy train:  0.955
training fold 7
Accuracy train:  0.96
training fold 8
Accuracy train:  0.967983991995998
training fold 9
Accuracy train:  0.9704852426213106
training fold 10
Accuracy train:  0.966983491745873
--------------

Accuracy train:  0.967032967032967
training fold 3
Accuracy train:  0.951048951048951
training fold 4
Accuracy train:  0.971
training fold 5
Accuracy train:  0.969
training fold 6
Accuracy train:  0.973
training fold 7
Accuracy train:  0.967
training fold 8
Accuracy train:  0.963963963963964
training fold 9
Accuracy train:  0.972972972972973
training fold 10
Accuracy train:  0.972972972972973
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9462020794298788
Mean f1 Train:  0.9479546392767161
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7657129316983591
 f1 test:  0.46851398930308813
----------------------------------------------------------
components 8
71
training fold 1
Accuracy train:  0.9660169915042479
training fold 2
Accuracy train:  0.9505247376311844
training fold 3
Accuracy train:  0.9685157421289355
training fold 4
Accuracy train:  0.9665
training fold 5
Accur

(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7150879687222321
 f1 test:  0.4168768186226964
----------------------------------------------------------
components 12
71
training fold 1
Accuracy train:  0.964035964035964
training fold 2
Accuracy train:  0.9630369630369631
training fold 3
Accuracy train:  0.9440559440559441
training fold 4
Accuracy train:  0.96
training fold 5
Accuracy train:  0.962
training fold 6
Accuracy train:  0.962
training fold 7
Accuracy train:  0.964
training fold 8
Accuracy train:  0.9579579579579579
training fold 9
Accuracy train:  0.9679679679679679
training fold 10
Accuracy train:  0.973973973973974
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9473118523954214
Mean f1 Train:  0.9490169732367563
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.783318523784136
 f1 test:  0.487516

# Basic Kernel Logistic Regresion (Polynomial)

In [22]:
plot = False
basicLR_stats_acc = []
basicLR_stats_f1 = []
res = []
for k in [1, 2, 3]:
    print('kernel '+str(k))
    for components in [2, 4, 6, 8, 10, 12, 14]:
        for size in[500, 1000, 10000, 20000, 50000]:
            print('components '+str(components))
            dataTrain = pd.read_csv(fileTrain)
            dataTest = pd.read_csv(fileTest)
            print(len(dataTrain.columns)) 
            try:
                dataTrain = dataTrain.loc[:, ~dataTrain.columns.str.contains('^Unnamed')]
                dataTrain =  dataTrain.drop(['Pred', 'Pred2', 'h', 'e', 'u','ID'], axis = 1)
                dataTrain = dataTrain.loc[:, (dataTrain != dataTrain.iloc[0]).any()]
                dataTest = dataTest[list(dataTrain.columns)]

            except:
                print('---')
            dataTrain = ut.downSampling(dataTrain)
            try:
                dataTrain = dataTrain.sample(size, random_state = 0)
            except:
                print('sample bigger that population')
            yTrain = 1*(dataTrain['label'] == 'ClassA')
            yTestHO = 1*(dataTest['label'] == 'ClassA')
            del dataTrain['label']
            del dataTest['label']

            names = dataTrain.columns
            scaler = preprocessing.StandardScaler()
            dataTrain = scaler.fit_transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain, columns=names)

            dataTest = scaler.fit_transform(dataTest)
            dataTest = pd.DataFrame(dataTest, columns=names)


            pca = PCA(n_components=components)
            pca.fit(dataTrain)
            dataTrain = pca.transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain)
            dataTrain = ut.kernelPolinomial(dataTrain,k)


            pca.fit(dataTest)
            dataTest = pca.transform(dataTest)
            dataTest = pd.DataFrame(dataTest)
            dataTest = ut.kernelPolinomial(dataTest,k)
            #size = dataTrain.shape[0]

            start_1 = timeit.default_timer()
            skf = StratifiedKFold(n_splits=int(10))
            skf.get_n_splits(dataTrain, yTrain)
            acc_kfold_Train = []
            f1_kfold_Train = []
            i = 0

            for train_index, test_index in skf.split(dataTrain, yTrain):
                        i = i+1
                        print('training fold ' + str(i))
                        X_train, X_test = dataTrain.iloc[train_index,:], dataTrain.iloc[test_index,:]
                        y_train, y_test = yTrain.iloc[train_index], yTrain.iloc[test_index]

                        basicLR = LogisticRegression(penalty='none', solver ='saga').fit(X_train, y_train)
                        predictions_1_Train = basicLR.predict(X_test)

                        accTrain = accuracy_score(y_test, predictions_1_Train, normalize=True)
                        f1Train = f1_score(y_test, predictions_1_Train, pos_label = 1)

                        if plot:
                            cm = confusion_matrix(y_test, predictions_1_Train)
                            ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')
                        print('Accuracy train: ', accTrain)
                        #print('Accuracy f1 Train: ', f1Train)
                        basicLR_stats_acc.append(accTrain)
                        basicLR_stats_f1.append(f1Train)
            accTrain = np.mean(basicLR_stats_acc)
            f1Train =np.mean(basicLR_stats_f1)
            print('----------------------------------------------------------')
            print('Summary Training')
            print('Mean Accuracy train: ', accTrain)
            print('Mean f1 Train: ', f1Train)
            stop_1 = timeit.default_timer()
            time_CV = stop_1 - start_1
            basicLR = LogisticRegression(penalty='none', solver ='saga').fit(dataTrain, yTrain)
            predictions_1_Test = basicLR.predict(dataTest)
            print(predictions_1_Test.shape)
            print(yTestHO.shape)
            print('----------------------------------------------------------')
            print('Summary Testing')
            accTest = accuracy_score(yTestHO, predictions_1_Test, normalize=True)
            f1Test = f1_score(yTestHO, predictions_1_Test,pos_label = 1)
            if plot: 
                cm = confusion_matrix(yTestHO, predictions_1_Test)
                ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')

            print(' Accuracy test: ', accTest)
            print(' f1 test: ', f1Test)
            print('----------------------------------------------------------')

            res.append([k, components, size,  accTrain, accTest, f1Train, f1Test])
            pd.DataFrame(res).to_csv('Results/summaryMCMC/'+'BasicCVAllDatawhitoutpenalty-'+str(number)+'.csv')

kernel 1
components 2
71
training fold 1
Accuracy train:  0.9215686274509803
training fold 2
Accuracy train:  0.8823529411764706
training fold 3
Accuracy train:  0.8627450980392157
training fold 4
Accuracy train:  0.9607843137254902
training fold 5
Accuracy train:  0.88
training fold 6
Accuracy train:  0.9
training fold 7
Accuracy train:  0.8163265306122449
training fold 8
Accuracy train:  0.8775510204081632
training fold 9
Accuracy train:  0.8163265306122449
training fold 10
Accuracy train:  0.8775510204081632
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.8795206082432973
Mean f1 Train:  0.8821269860917635
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6585036431491026
 f1 test:  0.36538969616908856
----------------------------------------------------------
components 2
71
training fold 1
Accuracy train:  0.900990099009901
training fold 2
Accuracy train:  0.92
trainin

(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7831171139150525
 f1 test:  0.48688193743693237
----------------------------------------------------------
components 6
71
training fold 1
Accuracy train:  0.9607843137254902
training fold 2
Accuracy train:  0.9411764705882353
training fold 3
Accuracy train:  1.0
training fold 4
Accuracy train:  0.9803921568627451
training fold 5
Accuracy train:  0.96
training fold 6
Accuracy train:  0.94
training fold 7
Accuracy train:  0.9387755102040817
training fold 8
Accuracy train:  0.9591836734693877
training fold 9
Accuracy train:  0.8571428571428571
training fold 10
Accuracy train:  0.9183673469387755
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9148848751073948
Mean f1 Train:  0.9166418510007557
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.720182453646111
 f1 tes

Accuracy train:  0.9545909181836367
training fold 9
Accuracy train:  0.9563912782556512
training fold 10
Accuracy train:  0.9467893578715744
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9315636411870324
Mean f1 Train:  0.9332382839649866
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7982820922931106
 f1 test:  0.5054606715464157
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.9411764705882353
training fold 2
Accuracy train:  0.9411764705882353
training fold 3
Accuracy train:  0.9803921568627451
training fold 4
Accuracy train:  0.9803921568627451
training fold 5
Accuracy train:  0.94
training fold 6
Accuracy train:  0.94
training fold 7
Accuracy train:  0.9387755102040817
training fold 8
Accuracy train:  0.9591836734693877
training fold 9
Accuracy train:  0.8775510204081632
training fold 10
Accuracy train:

Accuracy train:  0.9558088382323535
training fold 2
Accuracy train:  0.9622075584883023
training fold 3
Accuracy train:  0.9582083583283343
training fold 4
Accuracy train:  0.9658068386322736
training fold 5
Accuracy train:  0.965
training fold 6
Accuracy train:  0.9586
training fold 7
Accuracy train:  0.9597919583916783
training fold 8
Accuracy train:  0.9605921184236847
training fold 9
Accuracy train:  0.9617923584716943
training fold 10
Accuracy train:  0.9553910782156432
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9389901037995693
Mean f1 Train:  0.9406154296900137
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.8130324032936438
 f1 test:  0.5245970778731736
----------------------------------------------------------
components 14
71
training fold 1
Accuracy train:  0.9803921568627451
training fold 2
Accuracy train:  0.9215686274509803
training fold 3
Accuracy trai

(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.731520644511581
 f1 test:  0.4289206421209143
----------------------------------------------------------
components 2
71
training fold 1
Accuracy train:  0.8890221955608878
training fold 2
Accuracy train:  0.9004199160167966
training fold 3
Accuracy train:  0.8928214357128574
training fold 4
Accuracy train:  0.9034193161367726
training fold 5
Accuracy train:  0.8942
training fold 6
Accuracy train:  0.8954
training fold 7
Accuracy train:  0.8907781556311263
training fold 8
Accuracy train:  0.8999799959991999
training fold 9
Accuracy train:  0.8995799159831966
training fold 10
Accuracy train:  0.8915783156631326
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.936290327331188
Mean f1 Train:  0.9378500220143701
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7861264

...

Accuracy train:  0.9440279860069966
training fold 4
Accuracy train:  0.9555
training fold 5
Accuracy train:  0.9565
training fold 6
Accuracy train:  0.9445
training fold 7
Accuracy train:  0.9455
training fold 8
Accuracy train:  0.9549774887443722
training fold 9
Accuracy train:  0.9604802401200601
training fold 10
Accuracy train:  0.9484742371185593
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9365210143154034
Mean f1 Train:  0.9371129865628846
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7319708548071797
 f1 test:  0.42806219188471745
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.788242351529694
training fold 2
Accuracy train:  0.7916416716656669
training fold 3
Accuracy train:  0.8010397920415917
training fold 4
Accuracy train:  0.8058388322335532
training fold 5
Accuracy train:  0.8018
training fol

71
training fold 1
Accuracy train:  0.9465267366316842
training fold 2
Accuracy train:  0.9410294852573713
training fold 3
Accuracy train:  0.9400299850074962
training fold 4
Accuracy train:  0.951
training fold 5
Accuracy train:  0.9505
training fold 6
Accuracy train:  0.944
training fold 7
Accuracy train:  0.944
training fold 8
Accuracy train:  0.9539769884942472
training fold 9
Accuracy train:  0.9554777388694348
training fold 10
Accuracy train:  0.9469734867433717
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9327979375490244
Mean f1 Train:  0.9319353094626099
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7389965049463895
 f1 test:  0.433850740131579
----------------------------------------------------------
components 14
71
training fold 1
Accuracy train:  0.7020595880823836
training fold 2
Accuracy train:  0.7118576284743051
training fold 3
Accuracy train:  0.71

(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7576683845743736
 f1 test:  0.4575686856900393
----------------------------------------------------------
components 4
71
training fold 1
Accuracy train:  0.8395802098950524
training fold 2
Accuracy train:  0.8245877061469266
training fold 3
Accuracy train:  0.8510744627686156
training fold 4
Accuracy train:  0.854
training fold 5
Accuracy train:  0.8545
training fold 6
Accuracy train:  0.83
training fold 7
Accuracy train:  0.869
training fold 8
Accuracy train:  0.8374187093546773
training fold 9
Accuracy train:  0.8684342171085543
training fold 10
Accuracy train:  0.8419209604802401
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9205580961871452
Mean f1 Train:  0.9177897991767633
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.765973579764232
 f1 test:  0.4659

Accuracy train:  0.862
training fold 8
Accuracy train:  0.8368368368368369
training fold 9
Accuracy train:  0.8258258258258259
training fold 10
Accuracy train:  0.8388388388388388
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9114915521208107
Mean f1 Train:  0.9094895363099472
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6766068360879095
 f1 test:  0.3875975949026295
----------------------------------------------------------
components 8
71
training fold 1
Accuracy train:  0.8325837081459271
training fold 2
Accuracy train:  0.8315842078960519
training fold 3
Accuracy train:  0.8435782108945528
training fold 4
Accuracy train:  0.8395
training fold 5
Accuracy train:  0.8375
training fold 6
Accuracy train:  0.8095
training fold 7
Accuracy train:  0.8455
training fold 8
Accuracy train:  0.8354177088544272
training fold 9
Accuracy train:  0.8444222111055528
training fold 

71
training fold 1
Accuracy train:  0.8611388611388612
training fold 2
Accuracy train:  0.8311688311688312
training fold 3
Accuracy train:  0.8461538461538461
training fold 4
Accuracy train:  0.833
training fold 5
Accuracy train:  0.849
training fold 6
Accuracy train:  0.851
training fold 7
Accuracy train:  0.852
training fold 8
Accuracy train:  0.8358358358358359
training fold 9
Accuracy train:  0.8358358358358359
training fold 10
Accuracy train:  0.8548548548548549
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9015960523053692
Mean f1 Train:  0.9004285266957894
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6883123037734732
 f1 test:  0.3855855014246345
----------------------------------------------------------
components 12
71
training fold 1
Accuracy train:  0.8480759620189905
training fold 2
Accuracy train:  0.8445777111444278
training fold 3
Accuracy train:  0.84